In [11]:
# Install nbformat
import nbformat
from IPython import get_ipython

# Define the path to the notebook
notebook_path = "utils.ipynb"

# Read the notebook
with open(notebook_path, "r") as file:
    nb = nbformat.read(file, as_version=4)

# Execute cells
for cell in nb.cells:
    if cell.cell_type == "code":
        code = cell.source
        get_ipython().run_cell(code)


In [2]:
def read_water_gage_station(filename):
    df = pd.read_csv(
        filename,
        index_col="time",
    )
    return df


MeteostatStationLNFT5 = read_water_gage_station("MeteostatStationLNFT5.csv")
MeteostatStationKBYY0 = read_water_gage_station("MeteostatStationKBYY0.csv")
MeteostatStationKLBX0 = read_water_gage_station("MeteostatStationKLBX0.csv")


In [ ]:
# Dictionary mapping columns to their labels and units
column_labels = {
    "relative_humidity": r"Relative Humidity (%)",
    "total_precipitation": r"Total Precipitation (mm)",
    "air_temperature": r"Temperature (°C)",
    "average_wind_speed": r"Wind Speed (m/s)",
}


In [ ]:
base_path = os.getcwd()

# List of datasets
datasets = {
    "Meteostat_LNFT5": df_Meteostat_LNFT5,
    # "Meteostat_KBYY0": df_Meteostat_KBYY0,
    # "Meteostat_KLBX0": df_Meteostat_KLBX0
}

column_pairs = [
    ("wind_direction", "average_wind_speed"),
]


for subfolder, df in datasets.items():
    for direction_column, magnitude_column in column_pairs:
        title = f"{magnitude_column.capitalize()} vs. {direction_column.capitalize()} for {subfolder}"

        plot_windrose_facetgrid(
            df,
            direction_column=direction_column,
            magnitude_column=magnitude_column,
            title=title,
            base_path=base_path,
            subfolder=subfolder,
        )


In [ ]:
base_path = os.getcwd()

# List of datasets
datasets = {
    "Meteostat_LNFT5": df_Meteostat_LNFT5,
    # "Meteostat_KBYY0": df_Meteostat_KBYY0,
    # "Meteostat_KLBX0": df_Meteostat_KLBX0
}

# List of magnitude columns and their corresponding titles
columns = {
    "relative_humidity": "Relative Humidity",
    "total_precipitation": "Total Precipitation",
    "air_temperature": "Air Temperature",
    "average_wind_speed": "Average Wind Speed",
}

# Generate plots for each dataset and magnitude column
for subfolder, df in datasets.items():
    for col, title_part in columns.items():
        # Title for KDE, ECDF, and Time Series plots
        title = f"Annual Distribution of {title_part} for {subfolder}"

        # KDE Plot
        plot_kde_facetgrid(
            df,
            magnitude_column=col,
            title=title,
            base_path=base_path,
            subfolder=subfolder,
        )
        # ECDF Plot
        plot_ecdf_facetgrid(
            df,
            magnitude_column=col,
            title=title,
            base_path=base_path,
            subfolder=subfolder,
        )
        # Time Series Plot
        plot_timeseries_facetgrid(
            df,
            magnitude_column=col,
            title=f"Annual Time Series of {title_part} for {subfolder}",
            base_path=base_path,
            subfolder=subfolder,
        )


In [ ]:
def plot_kde(
                df, 
                direction_column, 
                magnitude_column, 
                title, 
                base_path=None, 
                subfolder=None
                ):
    if base_path and subfolder:
        full_path = os.path.join(base_path, subfolder)
        os.makedirs(full_path, exist_ok=True)
    else:
        full_path = base_path

    orinetation = 90 - 34
    df[direction_column] = (df[direction_column] - orinetation) % 360


    sns.set_theme(style="ticks")
    g = sns.JointGrid(
        data=df,
        x=direction_column,
        y=magnitude_column,
        marginal_ticks=True,
        space=0.5
    )
    g.fig.set_facecolor('white')
    cbar_ax = g.fig.add_axes([0.63, 0.15, 0.05, 0.2])  # x, y, width, height

    g.plot_joint(
        sns.kdeplot,
        log_scale=(False, True),
        fill=True,
        thresh=0,
        cut=0,
        levels=10,
        cbar=True,
        cbar_ax=cbar_ax
    )
    cbar_ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.3f}'))

    g.plot_marginals(
        sns.histplot,
        alpha=0.5,
        stat="probability",
        kde=True,
        bins=10,
        color="#03012d",
        element="step"
    )

    # Set y-axis label for the marginal histograms
    g.ax_marg_x.set_ylabel("Probability")
    g.ax_marg_y.set_ylabel("Probability")

    if magnitude_column == 'wave_height':
        ylabel = r"Wave Height (m)"
        xlabel = "Wave Direction (degree)"
        MultipleLocator = 1
    elif magnitude_column in ['wind_speed', 'average_wind_speed']:
        ylabel = r"Wind Speed (m/s)"
        xlabel = "Wind Direction (degree)"
        MultipleLocator = 2

    g.set_axis_labels(f"{xlabel}", f"{ylabel}")
    g.ax_joint.yaxis.set_major_locator(ticker.MultipleLocator(MultipleLocator))
    g.ax_joint.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:.0f}'.format(y)))
    g.ax_joint.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:.0f}'.format(x)))
    g.ax_joint.xaxis.set_major_locator(ticker.FixedLocator([0, 90, 180, 270, 360]))

    cm = 1/2.54  # centimeters in inches
    plt.gcf().set_size_inches(20*cm, 20*cm)  # Size in cm

    plt.suptitle(f'{title}', horizontalalignment='center', verticalalignment='top')
    
    
    # plt.suptitle(f'{title}\nYear: {year} - Month: {month}', horizontalalignment='center', verticalalignment='top')
    
    plt.tight_layout()
    
    save_path = os.path.join(full_path, f"{title}.svg") 
    
    # save_path = os.path.join(full_path, f"{title},Year {year},Month {month}.svg") if full_path else f"{title},Year {year},Month {month}.svg"
    plt.savefig(save_path, format='svg')
    plt.close(g.fig)

In [13]:
base_path = os.getcwd()

# List of datasets
datasets = {
    "Meteostat_LNFT5": MeteostatStationLNFT5,
    "Meteostat_KBYY0": MeteostatStationKBYY0 ,
    "Meteostat_KLBX0": MeteostatStationKLBX0
}

column_pairs = [
    ("wind_direction", "average_wind_speed"),
]

# Generate plots for each dataset and column pair
for subfolder, df in datasets.items():
    for direction_column, magnitude_column in column_pairs:
        # Title for the plot
        title = f"{magnitude_column.capitalize()} vs. {direction_column.capitalize()} for {subfolder}"

        # Year to focus on in the plot
        year = 2016  # Example year, modify as needed

        # KDE Plot
        plot_kde(
            df,
            direction_column=direction_column,
            magnitude_column=magnitude_column,
            year=year,
            title=title,
            base_path=base_path,
            subfolder=subfolder,
        )


/Users/abolfazlyousefi/anaconda3/lib/python3.10/site-packages/seaborn/axisgrid.py:1832: UserWarning: KDE cannot be estimated (0 variance or perfect covariance). Pass `warn_singular=False` to disable this warning.
  func(x=self.x, y=self.y, **kwargs)
/var/folders/ws/bm_885ln0hz8vlw6b_j_wy2m0000gn/T/ipykernel_45465/2076576609.py:87: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/abolfazlyousefi/anaconda3/lib/python3.10/site-packages/seaborn/axisgrid.py:1832: UserWarning: KDE cannot be estimated (0 variance or perfect covariance). Pass `warn_singular=False` to disable this warning.
  func(x=self.x, y=self.y, **kwargs)
/var/folders/ws/bm_885ln0hz8vlw6b_j_wy2m0000gn/T/ipykernel_45465/2076576609.py:87: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/abolfazlyousefi/anaconda3/lib/python3.10/site-packages/seab